In [ ]:
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, LLMMathChain
from langchain.agents import (
    get_all_tool_names,
    load_tools,
    Tool,
    initialize_agent
)

## Setting the LLM

In [ ]:
with open("openai_api.txt", "r") as f:
    OPENAI_API = f.read()

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    temperature = 0,
    openai_api_key = OPENAI_API
)

## Need of Agents

Some applications require not just a predetermined chain of calls to LLM, but potentially an `unknown chain` that depends on the user's input. In these type of chains, there is an `agent` which has access to a suite of `tools`. Thus depending on the user's input, the agent then decides which, if any, of these tools to call.

There are two main types of agents:
1. **Action Agents** - agents that decides the actions to take and execute those, one at a time.
2. **Plan-and-Execute Agents** - agents that decides a plan of actions to take, and execute those one at a time.

`Action Agents` are more conventional, and good for small tasks. For more complex tasks, or long running tasks, the initial planning steps helps to maintain long term objectives and focus, thus `Plan-and-Execute Agents` are more suitable. However that comes of generally more calls and highet latency.


## Action Agents

High level pseudocode of Action Agents:
1. `Receive` User input.
2. `Decide` which **tool** to use, if any, and what the **tool input** should be.
3. `Call` the **tool** with the **tool input**, and `record` an **observation** (the output of this calling).
4. `Decide` the next step according the **tool**, **tool input** and **observation**.

-

To better understand Agents let's see the abstraction that are involed:
* `Agents`: An interface that takes in user input along side with a list of previous steps, apply "logic" and return either an AgentAction or AgentFinish.
    * `AgentAction` - it's a dataclass that represents the action an agent should take. It has a tool property (which is the name of the tool that should be invoked) and a tool_input property (the input to that tool)
    * `AgentFinish` - it's a dataclass that signifies that the agent has finished and should return to the user. It has a return_values parameter, which is a dictionary to return. It often only has one key - output - that is a string, and so often it is just this key that is returned

* `Tools`: They're functions that an agent calls. There are two important considerations here:
    1. Giving the agent access to the right tools
    2. Describing the tools in a way that is most helpful to the agent

* `Toolkits`: Often the set of tools an agent has access to is more important than a single tool. For this LangChain provides the concept of toolkits - groups of tools needed to accomplish specific objectives. There are generally around 3-5 tools in a toolkit.

* `Agent Executor`: The agent executor is the runtime for an agent. This is what actually calls the agent and executes the actions it chooses. Pseudocode for this runtime is below:
```
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action
```

## Agents

In order to load agents, we should understand the following concepts:
1. `Tool`: A function, a Chain or even another agent, that performs a specific duty, such as Google Search, Database lookup, etc.
2. `LLM`: The language model powering the agent.
3. `Agent`: The type of agent to use.


In [ ]:
## The Calculator Tool
# When initializing tools, we either create a custom tool or load a pre-built tool. In either cases the `tool` is a `utility chain` with a `name` and `description`

# Initializing the Math Chain (we should pass an LLM Chain instead of an LLM)
llm_math = LLMMathChain.from_llm(llm=llm)

# Initializing the Custom Tool
math_tool = Tool(
    name = "Calculator",
    description = "Useful for math questions.",
    func = llm_math.run
)

# When passing tools to LLM we should pass them as List
tools = [math_tool]

In [ ]:
tools[0].name, tools[0].description

In [ ]:
## We can see all tools available using:

get_all_tool_names()

In [ ]:
## Loading a tool

tools = load_tools(
    tool_names = ["llm-math"],
    llm = llm
)

In [ ]:
tools[0].name, tools[0].description

We now have the LLM and tools but no agent. To initialize a simple agent, we can do the following:

In [ ]:
zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description", # the type of the agent
    tools = tools,
    llm = llm,
    verbose = True, # for printing what model is performing each query
    max_iterations = 3
)

## Available Agent types can be found on `langchain.agents.agent_types.py`

`Zero-shot` means the agent functions on the current action only — it has `no memory`.

In [ ]:
## Using the Agent

zero_shot_agent("what is (4.5*2.1)^2.2?")

In [ ]:
(4.5*2.1)**2.2

In [ ]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")

In [ ]:
zero_shot_agent("what is the capital of Greece?")

We run into an error. The problem here is that the agent keeps trying to use a tool. Yet, our agent contains only one tool — the calculator. So by giving the `Agent` more `Tools` we can solve this problem.

In [ ]:
## Setting Prompt Template and LLM Chain

prompt = PromptTemplate(
    template = "{query}",
    input_variables = ["query"]
)

llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

In [ ]:
## Initializing the LLM Tool

llm_tool = Tool(
    name = "Language Model",
    description = "Use this tool for general purpose queries and logic.",
    func = llm_chain.run
)

In [ ]:
## Adding the Tool above to `tools`

tools.append(llm_tool)

In [ ]:
## Reinitializing the Agent

zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iteations = 3
)

In [ ]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

In [ ]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")

In [ ]:
zero_shot_agent("what is the capital of Greece?")

The format of 'reasoning' is the following:
* `Question` - the input question you must answer
* `Thought` - the LLM should always think about what to do
* `Action` - take action using one of the tools
* `Action Input` - the input to the action
* `Observation` - the result of the action
* `Repeat` - repeat thought/action/action input/observation until reaches a result
* `Thought` - now know the final answer

In [ ]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

These `tools` and the `thought process` separate agents from chains in LangChain.

Whereas a `chain` defines an `immediate` input/output process, the logic of `agents` allows a `step-by-step` thought process. The advantage of this step-by-step process is that the LLM can work through multiple reasoning steps or tools to produce a better answer.

-

The final aspect we need to understand is `Thought:{agent_scratchpad}`.

The `agent_scratchpad` is where we add every thought or action the agent has already performed. All thoughts and actions (within the current agent executor chain) can then be accessed by the next thought-action-observation loop, enabling continuity in agent actions.